In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control as ct
from pytz import timezone

import sys

sys.path.append("../../../")

from utils.plot import (
    config_matplotlib,
    figsize,
    fig_save_and_show,
    plot_events_data,
)
from utils.optimization import (
    save_model_params_to_json,
    load_model_params_from_json,
)
from utils.data import (
    export_dataframe_to_latex,
)
from utils.sensitivity_analysis import (
    describe_param_with_uniform_distribution,
    describe_param_with_log_uniform_distribution,
    describe_param_with_triangular_distribution,
    describe_param_with_normal_distribution,
    describe_param_with_truncated_normal_distribution,
    describe_param_with_log_normal_distribution,
    wrapped_model_function,
    create_problem_spec_and_sample,
    plot_sampled_distribuitions,
    evaluate_problem,
    analyze_time_step,
    analyze_problem,
    sobol_sensitivity_analysis_from_model_params_to_outputs,
    plot_sensitivity_analysis,
    plot_sensitivity_analysis_heatmaps,
    get_region_mean,
    plot_sensitivity_analysis_bars,
)
from utils.models import eval_poly, lut_interp

import warnings

warnings.filterwarnings("ignore")

config_matplotlib()
latex_img_path = "/home/joaoantoniocardoso/workspace_TCC/repo/thesis/assets/generated/"
latex_tex_path = "/home/joaoantoniocardoso/workspace_TCC/repo/thesis/tex/generated/"

# Sensitivity Analysis


In [ ]:
from model import Battery1RCDeepGas as Battery

Battery.build({})

In [ ]:
batt_name = "D35"
cell_amps_hour = 48
series_cells = 3
parallel_cells = 1
cell_voltage = 12
nominal_Q_Ah = parallel_cells * cell_amps_hour  # Capacity, in Amper-hour
batt_Q = nominal_Q_Ah * 3600  # [Ah] -> [As]
nominal_voltage = series_cells * cell_voltage
ocv_params = load_model_params_from_json('battery_ocv_params.json')
# batt_k_V_OC_coeffs = ocv_params['batt_k_V_OC_coeffs']
batt_ocv_lut = ocv_params['batt_ocv_lut']
initial_SOC = 1.2

battery_params = dict(
    # Capacity / efficiencies
    batt_Q=0.85 * batt_Q,  # 172800 C per 12 V block
    batt_eta_dis=1.0,
    batt_eta_chg=0.8,
    # Ohmic and dynamic branch
    batt_R_0=0.003,
    batt_R_1=0.01,
    batt_R_2=0.01,
    batt_C_1=1e1 / 0.01,
    batt_C_2=1e3 / 0.01,
    # Deep-discharge shaping
    batt_z_deep=0.3,
    batt_R_0_deep=0.05,
    batt_beta_deep=10,
    batt_k_deep=0.05,
    # Overcharge / gassing
    batt_z_gas=1.2,
    batt_alpha_gas=20,
    batt_k_ov_in=0.1,
    batt_tau_ov=5.0,
    batt_V_ov_max=3.0,
    batt_q_ov_ref=1.5,
    # OCV polynomial (per block)
    batt_ocv_lut=batt_ocv_lut,  # [10.5, 13.0],
    # Series blocks
    batt_N_S=3,
)

model_params = battery_params
model_params


In [ ]:
def _estimate_open_circuit_voltage(params, batt_z):
    if "batt_ocv_lut" in params:
        return float(lut_interp(params["batt_ocv_lut"], batt_z))
    if "batt_k_V_OC_coeffs" in params:
        coeffs = params["batt_k_V_OC_coeffs"]
        try:
            return float(lut_interp(coeffs, batt_z))
        except Exception:
            return float(eval_poly(coeffs, batt_z))
    return 0.0


def model_function(T, U, X0, **params):
    return ct.input_output_response(
        Battery.build(params=params),
        T=T,
        U=U,
        X0=X0,
        solve_ivp_method="Radau",
    ).to_pandas()


def battery_sensitivity_analysis_step_response(
    params_description: dict,
    current_step: float,
    samples: int,
    do_plot=True,
):
    model_class = Battery
    model_tmp = model_class.build(model_params)

    T = np.linspace(0, 200, 201, endpoint=True)

    U = np.empty([model_tmp.ninputs, len(T)]) * np.nan
    U[model_tmp.input_labels.index("batt_i")] = current_step

    batt_z_0 = 0.9
    batt_v_0 = model_params.get("batt_N_S", 1) * _estimate_open_circuit_voltage(
        model_params, batt_z_0
    )

    X0 = model_class.initial_state(
        X0={
            "batt_z": batt_z_0,
            "batt_v": batt_v_0,
        },
        U0=U[:, 0],
        params=model_params,
    )

    df_sa, problem = sobol_sensitivity_analysis_from_model_params_to_outputs(
        params_description=params_description,
        T=T,
        U=U,
        X0=X0,
        model_function=model_function,
        model_class=model_class,
        model_params=model_params,
        samples=samples,
        calc_second_order=False,
        seed=42,
        nprocs=32,
    )

    df_steady_state_mean = get_region_mean(df_sa, t_start=180, t_end=200)

    export_dataframe_to_latex(
        filename=f"{latex_tex_path}/PNGV_1rc_deep_and_gas_sensitivity_analysis_{current_step}.tex",
        label=f"table:PNGV_1rc_deep_and_gas_sensitivity_analysis_{current_step}",
        caption="Índices de Sobol em regime permanente",
        df_steady_state_mean=df_steady_state_mean,
    )

    if do_plot:
        display(df_steady_state_mean)

        fig_save_and_show(
            filename=f"{latex_img_path}/PNGV_1rc_deep_and_gas_sensitivity_analysis_distributions.pdf",
            save_title="Distribuição dos valores para cada fator",
            show_title="Distribution of the values for each factor",
            fig=plot_sampled_distribuitions(problem),
            ncols=3,
        )

        for output in df_sa["output"].unique():
            fig_save_and_show(
                filename=f"{latex_img_path}/PNGV_1rc_deep_and_gas_sensitivity_analysis_{current_step}_{output}.pdf",
                save_title=f"Índices de Sobol para a saída {output} durante o tempo, com degrau de {current_step} [A], com {samples} amostras",
                show_title=f"Sobol Indices for the output {output} over time with step of {current_step} [A], with {samples} samples",
                fig=plot_sensitivity_analysis(df_sa, output=output),
                ncols=3,
            )

        for output in df_sa["output"].unique():
            fig_save_and_show(
                filename=f"{latex_img_path}/PNGV_1rc_deep_and_gas_sensitivity_analysis_heatmaps_{current_step}_{output}.pdf",
                save_title=f"Índices de Sobol para a saída {output} durante o tempo, com degrau de {current_step} [A], com {samples} amostras",
                show_title=f"Sobol Indices for the output {output} over time with step of {current_step} [A], with {samples} samples",
                fig=plot_sensitivity_analysis_heatmaps(df_sa, output=output),
                ncols=3,
            )

        for output in df_sa["output"].unique():
            fig_save_and_show(
                filename=f"{latex_img_path}/PNGV_1rc_deep_and_gas_sensitivity_analysis_bars_{current_step}_{output}.pdf",
                save_title=f"Índices de Sobol para a saída {output} em regime permanente, com degrau de {current_step} [A], com {samples} amostras",
                show_title=f"Sobol Indices for the output {output} at steady-state with step of {current_step} [A], with {samples} samples",
                fig=plot_sensitivity_analysis_bars(df_steady_state_mean, output=output),
                ncols=3,
            )

    return df_sa, problem

In [ ]:
# Setting all uncertainties to be x%
u_batt_R_0 = 1 / 100
u_batt_R_1 = 1 / 100
u_batt_R_2 = 1 / 100
u_batt_C_1 = 1 / 100
u_batt_C_2 = 1 / 100
u_batt_Q = 1 / 100
u_batt_eta = 1 / 100

In [ ]:
%%time

params_description = {
    'batt_Q': describe_param_with_uniform_distribution(lower=model_params['batt_Q']*(1-u_batt_Q), upper=model_params['batt_Q']*(1+u_batt_Q)),
    'batt_R_0': describe_param_with_uniform_distribution(lower=model_params['batt_R_0']*(1-u_batt_R_0), upper=model_params['batt_R_0']*(1+u_batt_R_0)),
    'batt_R_1': describe_param_with_uniform_distribution(lower=model_params['batt_R_1']*(1-u_batt_R_1), upper=model_params['batt_R_1']*(1+u_batt_R_1)),
    'batt_C_1': describe_param_with_uniform_distribution(lower=model_params['batt_C_1']*(1-u_batt_C_1), upper=model_params['batt_C_1']*(1+u_batt_C_1)),
    'batt_R_2': describe_param_with_uniform_distribution(lower=model_params['batt_R_2']*(1-u_batt_R_2), upper=model_params['batt_R_2']*(1+u_batt_R_2)),
    'batt_C_2': describe_param_with_uniform_distribution(lower=model_params['batt_C_2']*(1-u_batt_C_2), upper=model_params['batt_C_2']*(1+u_batt_C_2)),
    'batt_eta_dis': describe_param_with_uniform_distribution(lower=model_params['batt_eta_dis']*(1-u_batt_eta), upper=model_params['batt_eta_dis']*(1+u_batt_eta)),
    'batt_eta_chg': describe_param_with_uniform_distribution(lower=model_params['batt_eta_chg']*(1-u_batt_eta), upper=model_params['batt_eta_chg']*(1+u_batt_eta)),
}

df_sa, problem = battery_sensitivity_analysis_step_response(
    params_description=params_description,
    current_step=150,
    samples=2**8,
    do_plot=True,
)


df_sa, problem = battery_sensitivity_analysis_step_response(
    params_description=params_description,
    current_step=-30,
    samples=2**8,
    do_plot=True,
)

